In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error,explained_variance_score
from sklearn.preprocessing import PolynomialFeatures

import wrangle as w

import model as m
import env

import warnings
warnings.filterwarnings("ignore")

# Acquire/Prep

In [2]:
df = w.get_zillow_data()

In [3]:
df= w.zillow_prep(df)

In [4]:
train,validate,test = w.split(df)

In [5]:
train, test, validate, train_scaled, validate_scaled, test_scaled = m.create_cluster('loc_clusters', train,validate, test,['longitude', 'latitude','tax_value'],5)
train, test, validate, train_scaled, validate_scaled, test_scaled = m.create_cluster('cluster_price_size', train,validate, test,['taxamount', 'sqft', 'lot_sqft'],4)
train, test, validate, train_scaled, validate_scaled, test_scaled = m.create_cluster('cluster_delinquency_value', train,validate, test,['tax_value', 'sqft','lot_sqft','has_taxdelinquency'],4)


# Explore

# Model

In [6]:
features_to_model1 =['has_taxdelinquency', 'cluster_price_size_0', 'cluster_price_size_1', 'age','log_error']


In [7]:
features_to_model2= ['loc_clusters','log_error']


In [8]:
# prepare data for modeling features set 1
X_train_scaled,y_train, X_validate_scaled,y_validate, X_test_scaled, y_test = m.model_data_prep(train_scaled, validate_scaled,test_scaled, features_to_model1)



In [9]:
# prepare data for modeling features set 2
X_train_scaled2,y_train2, X_validate_scaled2,y_validate2, X_test_scaled2, y_test2 = m.model_data_prep2(train_scaled, validate_scaled,test_scaled, features_to_model2)



In [10]:
# features set 1 on train and validate
metric1= m.modeling(X_train_scaled, y_train, X_validate_scaled,y_validate, X_test_scaled, y_test)


Features set 1
RMSE for F1: OLS using LinearRegression
Training/In-Sample:  0.17129603784721747 
Validation/Out-of-Sample:  0.16050281798358967
_______________
RMSE for Polynomial Model, degrees=2
Training/In-Sample:  0.1712784324311459 
Validation/Out-of-Sample:  0.1604820218870208


In [11]:
# features set 2 on train and validate
metric2 = m.modeling2(X_train_scaled2, y_train2, X_validate_scaled2,y_validate2, X_test_scaled2, y_test2)

Features Set 2
RMSE for F2: OLS using LinearRegression
Training/In-Sample:  0.17138204421880615 
Validation/Out-of-Sample:  0.16050657377284286
_______________
RMSE for F2: Polynomial Model, degrees=2
Training/In-Sample:  0.17138197656112683 
Validation/Out-of-Sample:  0.1605061869241377


In [12]:
metric=pd.concat([metric1,metric2],ignore_index=True)
metric

,model,RMSE_validate
0,mean_baseline,0.171387
1,F1: OLS Regressor,0.160503
2,F1: degree2,0.160482
3,mean_baseline,0.171387
4,F2: OLS Regressor,0.160507
5,F2: degree2,0.160506


#TEST F2:OLS REGRESSOR.......

In [14]:
X_train_scaled2,y_train2, X_validate_scaled2,y_validate2, X_test_scaled2, y_test2 = m.model_data_prep(train_scaled, validate_scaled,test_scaled, features_to_model2)

In [15]:
y_test

,log_error
33007,-0.127641
11075,-0.416235
14604,0.062431
39932,0.034278
13340,0.007982
...,...
27073,0.021019
11634,-0.035492
45815,0.000572
27328,0.021946


# Test Evaluation

In [16]:
m.test_modeling2(X_train_scaled2, y_train2, X_validate_scaled2,y_validate2, X_test_scaled2, y_test2)

,model,RMSE_validate,r^2_validate
0,mean_baseline,0.171387,0.000000
1,Train F2: degree2,0.171382,0.000053
2,Validate F2: degree2,0.160507,0.000226
3,Test F2: degree2,0.182255,-0.000112
